# 합성곱신경망(Convolutional Neural Network,CNN)

## CNN의 주요 개념과 구조

**이미지 특화된 CNN**
CNN은 이미지나 영상 데이터 같은 다차원 데이터를 효과적으로 처리하기 위해 설계된 인공 신경망의 일종이다. 합성곱 연산을 통해 입력 데이터에서 패턴을 추출하고, 이를 기반으로 분류나 예측을 수행한다.(`CNN`은 **출력 layer에는 적합하지 않기 때문에, 마지막에 완전 연결층을 출력 layer로 두며 ANN을 활용한다**)

1. Layer

- 합성곱 층 (Convolutional Layer): 

    - 입력 데이터에 **필터(또는 커널)**를 적용하여 **특징 맵(feature map)**을 생성한다.
    - 합성곱 층은 데이터에서 지역적인 패턴을 학습하며, 필터의 크기와 개수를 통해 다양한 특징을 추출한다.
    - `nn.Conv2d` 함수를 사용하여 합성곱 층을 정의한다.
  
        ```nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)```
        
        - `in_channels`: 입력 이미지의 채널 수. 예를 들어, 흑백 이미지의 경우 1, 컬러 이미지의 경우 3이다.
        - `out_channels`: 필터의 개수, 즉 출력 특징 맵의 채널 수이다.
        - `kernel_size`: 필터의 크기. 예를 들어 (3, 3) 또는 3으로 정의할 수 있다.
        - `stride`: 필터가 이동하는 간격을 의미한다.
        - `padding`: 입력 이미지의 가장자리를 감싸는 패딩의 크기이다. 패딩을 통해 출력 크기를 조정할 수 있다.

- 풀링 층 (Pooling Layer):

    - 특징 맵의 크기를 줄여 계산량을 줄이고, 오버피팅을 방지한다. 주로 Max Pooling(최대값)과 Average Pooling(평균값)이 사용된다.
    - 풀링 층은 Local 특징 중에서 중요한 값만 유지한다.
    - **nn.MaxPool2d**를 사용해 풀링을 정의할 수 있다.
        
        ```nn.MaxPool2d(kernel_size, stride=None, padding=0)```
        - `kernel_size`: 풀링 필터의 크기이다.
        - `stride`: 풀링 필터의 이동 간격이다. 기본적으로는 kernel_size와 동일하다.
        - `padding`: 풀링 전에 입력 데이터에 패딩을 추가할 수 있다.

- 완전 연결 층 (Fully Connected Layer):

    - 합성곱과 풀링 과정을 통해 추출된 특징을 바탕으로 최종적으로 분류 작업을 수행한다.
    - **nn.Linear**를 사용해 층을 정의한다.
    
2. 활성화 함수 (Activation Function)

    활성화 함수는 각 층에서 비선형성을 추가하기 위해 사용된다. CNN에서는 주로 **ReLU (Rectified Linear Unit)**가 사용된다. 이는 음수 값을 0으로 변환하여 네트워크에 비선형 특성을 부여한다.
    
    - ReLU 함수 적용 방법:
    
    ```python
        import torch.nn.functional as F

        x = F.relu(x)
```
        F.relu()는 입력 값 x의 음수를 모두 0으로 변환한다.
        
3. CNN의 전체 구성 및 흐름

예시:

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 첫 번째 합성곱 층 정의: 입력 채널 1, 출력 채널 6, 커널 크기 3x3
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 두 번째 합성곱 층 정의: 입력 채널 6, 출력 채널 16, 커널 크기 3x3
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 완전 연결 층 정의
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 첫 번째 합성곱 층과 ReLU, 그리고 Max Pooling 적용
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        # 두 번째 합성곱 층과 ReLU, 그리고 Max Pooling 적용
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        # 데이터의 차원 변환 (Flatten): (배치 크기, -1)
        x = x.view(-1, 16 * 6 * 6)
        # 완전 연결 층을 통과시키며 ReLU 적용
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 최종 출력 층
        x = self.fc3(x)
        return x

# 모델 인스턴스 생성
model = SimpleCNN()
```

- 합성곱과 활성화: 입력 데이터를 필터로 감지하며 각 필터는 특정 특징을 추출한다. conv1, conv2와 같은 합성곱 층을 통과하면서 점점 더 복잡한 특징이 추출된다.
- 풀링: Max Pooling을 사용하여 데이터 크기를 줄여 중요한 정보만 남긴다.
- Flatten: 마지막 합성곱 층에서 나온 결과를 완전 연결 층에 넣기 위해 1차원 벡터 형태로 변환한다.
- x.view(-1, 16 * 6 * 6)에서 -1은 배치 크기를 자동으로 계산하는 역할을 한다.
- 완전 연결 층과 분류: 마지막 단계에서 추출된 특징 벡터를 이용해 최종적으로 각 클래스에 대한 확률 값을 예측한다.



***

CNN은 이미지 데이터에서 특징을 자동으로 추출하여 이를 바탕으로 예측을 수행하는 딥러닝 모델이다. 합성곱 층에서 특징을 추출하고, 풀링 층에서 크기를 줄이며, 마지막으로 완전 연결 층을 통해 예측을 수행하는 것이 CNN의 주요 학습 흐름이다. 이러한 구조 덕분에 CNN은 이미지 분류, 객체 탐지 등 컴퓨터 비전 문제에서 큰 성과를 내고 있다.

## 실습

In [1]:
import torch  # PyTorch의 기본 기능 제공 (텐서 연산, 자동 미분 등)
import torch.nn as nn  # 신경망 네트워크 생성에 필요한 클래스와 함수 제공
import torch.optim as optim  # 최적화 알고리즘 (SGD, Adam 등) 제공
import torchvision  # 이미지 데이터셋, 모델, 변환 등을 포함한 컴퓨터 비전 라이브러리
import torchvision.transforms as transforms  # 이미지 데이터 전처리를 위한 다양한 변환 함수 제공

In [2]:
 # 데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환 (픽셀 값의 범위가 [0, 1]로 스케일링됨). 이 작업은 이미지 데이터를 파이토치 모델이 처리할 수 있는 형태로 변환함
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 각 채널(R, G, B)에 대해 평균 0.5, 표준편차 0.5로 정규화. 이는 데이터의 범위를 [-1, 1]로 변환하여 학습을 안정화하고, 신경망이 더 빠르게 수렴하도록 돕는 역할을 함
])

# CIFAR-10 데이터셋 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)  # CIFAR-10 학습 데이터셋 로드, 필요 시 다운로드, 전처리 적용
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)  # 학습 데이터셋을 배치 크기 64로 나누고, 랜덤하게 섞어서 불러옴

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)  # CIFAR-10 테스트 데이터셋 로드, 필요 시 다운로드, 전처리 적용
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)  # 테스트 데이터셋을 배치 크기 64로 나누고, 순차적으로 불러옴

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# SimpleCNN 클래스 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # 첫 번째 합성곱 층: 입력 채널 3 (RGB 이미지), 출력 채널 32, 커널 크기 3x3, 패딩 1 (출력 크기를 입력과 동일하게 유지)
        self.pool = nn.MaxPool2d(2, 2)  # 최대 풀링 층: 풀링 크기 2x2, 스트라이드 2 (입력의 크기를 절반으로 줄임)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 두 번째 합성곱 층: 입력 채널 32, 출력 채널 64, 커널 크기 3x3, 패딩 1 (출력 크기를 입력과 동일하게 유지)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)  # 완전 연결 층: 입력 차원 64*8*8 (Conv2 이후의 출력 크기 플래튼), 출력 차원 512
        self.fc2 = nn.Linear(512, 10)  # 출력 층: 10개의 클래스 (CIFAR-10의 클래스 수)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # 첫 번째 합성곱 층 -> ReLU 활성화 함수 -> 최대 풀링 적용
        x = self.pool(torch.relu(self.conv2(x)))  # 두 번째 합성곱 층 -> ReLU 활성화 함수 -> 최대 풀링 적용
        x = x.view(-1, 64 * 8 * 8)  # 텐서를 플래튼(flatten)하여 완전 연결 층에 전달 (배치 크기는 유지하며 나머지 차원을 일렬로 펼침)
        x = torch.relu(self.fc1(x))  # 첫 번째 완전 연결 층 -> ReLU 활성화 함수 적용
        x = self.fc2(x)  # 두 번째 완전 연결 층 (출력 층)
        return x

In [4]:
# 모델 초기화
model = SimpleCNN()  # SimpleCNN 클래스의 인스턴스 생성 (합성곱 신경망 모델)

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()  # 분류 문제에 사용되는 손실 함수로, 모델의 예측 값과 실제 레이블 값 사이의 차이를 계산
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # 경사 하강법(SGD)을 통해 모델의 파라미터를 최적화 (lr: 학습률, momentum: 학습 가속도)

# 모델 학습
epochs = 10  # 전체 데이터셋을 10번 반복 학습 (에포크 수)
for epoch in range(epochs):
    running_loss = 0.0  # 에포크 동안 손실값을 누적하여 로그를 남기기 위함
    
    # 데이터 로더를 통해 배치 단위로 데이터 불러오기
    for i, data in enumerate(trainloader, 0):  # trainloader에서 데이터를 배치 단위로 불러오며 인덱스도 함께 가져옴 (i는 인덱스)
        inputs, labels = data  # 입력 데이터와 실제 레이블을 data에서 분리

        # 기울기 초기화 (이전 배치의 기울기를 제거)
        optimizer.zero_grad()

        # 순전파 + 손실 계산 + 역전파 + 최적화
        outputs = model(inputs)  # 입력 데이터를 모델에 통과시켜 예측 값(outputs) 생성
        loss = criterion(outputs, labels)  # 예측 값과 실제 레이블 간 손실 계산
        loss.backward()  # 역전파를 통해 각 파라미터에 대한 기울기 계산 (손실 값을 기준으로 파라미터 업데이트 방향을 계산)
        optimizer.step()  # 기울기를 바탕으로 가중치 업데이트 (파라미터 최적화)

        # 손실 출력 (중간중간 손실 로그를 남겨 학습 상황을 모니터링)
        running_loss += loss.item()  # 현재 배치의 손실 값을 running_loss에 누적
        if i % 100 == 99:  # 매 100 미니배치마다 손실 값을 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')  # 에포크와 배치 번호를 포함해 평균 손실 값 출력
            running_loss = 0.0  # 로그 출력 후 손실 값 초기화

print('Finished Training') 

[Epoch 1, Batch 100] loss: 2.127
[Epoch 1, Batch 200] loss: 1.786
[Epoch 1, Batch 300] loss: 1.603
[Epoch 1, Batch 400] loss: 1.484
[Epoch 1, Batch 500] loss: 1.426
[Epoch 1, Batch 600] loss: 1.367
[Epoch 1, Batch 700] loss: 1.330
[Epoch 2, Batch 100] loss: 1.186
[Epoch 2, Batch 200] loss: 1.159
[Epoch 2, Batch 300] loss: 1.114
[Epoch 2, Batch 400] loss: 1.103
[Epoch 2, Batch 500] loss: 1.077
[Epoch 2, Batch 600] loss: 1.010
[Epoch 2, Batch 700] loss: 1.007
[Epoch 3, Batch 100] loss: 0.891
[Epoch 3, Batch 200] loss: 0.902
[Epoch 3, Batch 300] loss: 0.873
[Epoch 3, Batch 400] loss: 0.857
[Epoch 3, Batch 500] loss: 0.854
[Epoch 3, Batch 600] loss: 0.843
[Epoch 3, Batch 700] loss: 0.838
[Epoch 4, Batch 100] loss: 0.685
[Epoch 4, Batch 200] loss: 0.711
[Epoch 4, Batch 300] loss: 0.715
[Epoch 4, Batch 400] loss: 0.676
[Epoch 4, Batch 500] loss: 0.693
[Epoch 4, Batch 600] loss: 0.707
[Epoch 4, Batch 700] loss: 0.690
[Epoch 5, Batch 100] loss: 0.541
[Epoch 5, Batch 200] loss: 0.527
[Epoch 5, 

In [5]:
# 모델 평가
correct = 0  # 올바르게 예측한 샘플 수
total = 0  # 전체 샘플 수
with torch.no_grad():  # 평가 단계에서는 기울기를 계산하지 않음 (메모리 절약 및 속도 향상)
    for data in testloader:  # 테스트 데이터 로더를 통해 데이터 반복
        images, labels = data  # 입력 이미지와 실제 레이블을 분리
        outputs = model(images)  # 모델을 통해 예측 값(outputs) 생성
        _, predicted = torch.max(outputs.data, 1)  # 각 샘플에 대해 가장 높은 값(확률)을 가지는 클래스 예측
        total += labels.size(0)  # 전체 샘플 수 누적
        correct += (predicted == labels).sum().item()  # 올바르게 예측한 샘플 수 누적


print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')  # 전체 테스트 데이터에 대한 모델의 정확도 출력


Accuracy of the network on the 10000 test images: 72.47%
